# TensorBoardによる単層NNのネットワークグラフの確認

In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(42)
tf.set_random_seed(42)

mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### - with構文を用いたグラフコンテキスト内に、Placeholder, Variable, 計算値の定義を記載する
### - with構文によるネームスコープを用いて、入力層、隠れ層、出力層などに構成要素をグループ化する
### - ネットワークグラフに付与するラベル名をコード内で指定する
### - グラフに表示するパラメーターを宣言して SummaryWriterオブジェクトでデータを書き出す

In [24]:
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default(): #with構文は、開始と終了に必ずしなければいけない一連の作業がある場合に活躍する
            self.prepare_model(num_units)
            self.prepare_session()
    
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]), name='weights')
            b1 = tf.Variable(tf.zeros([num_units]), name = 'biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) +b1, name='hidden1')
            
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = - tf.reduce_sum(t*tf.log(p), name = 'loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')
            
        tf.summary.scalar("loss", loss)  #tf.scalar_summary()ではなくtf.summary.scalar()を使う
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("weights_hidden", w1) # tf.histogram_summary()ではなくtf.summary.histogram()を使う
        tf.summary.histogram("biases_hidden", b1)
        tf.summary.histogram("weights_output", w0)
        tf.summary.histogram("biases_output", b0)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        summary = tf.summary.merge_all() #それぞれのサマリーをマージしている tf.merge_all_summaries()ではなくtf.summary.merge_all()を使う
        writer = tf.summary.FileWriter("./mnist_sl_logs", sess.graph) #tf.train.SummaryWriter() -> tf.summary.FileWriter()
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

with構文によって、個別のネームスコープのコンテキストを設定している。
これは、それぞれの構成要素をグループ化するもので、ネットワークグラフを表示する際に、同じグループの要素が1つの枠にまとめて表示される。
with構文でネームスコープを設定する場合は、引数でグループの名前を指定する。また、それぞれの要素に対して、nameオプションで、ネットワーク上に表示する名前を指定している。

### データ出力先のディレクトリに以前のデータが残っていると出力が乱れるのでディレクトリを削除しておく

In [11]:
!rm -rf ./mnist_sl_logs

In [25]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x:batch_xs, nn.t: batch_ts})
    if i % 100 ==0:
        summary, loss_val, acc_val = nn.sess.run([nn.summary, nn.loss, nn.accuracy], feed_dict = {nn.x:mnist.test.images, nn.t:mnist.test.labels})
        print('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

Step: 100, Loss: 2541.569336, Accuracy: 0.925900
Step: 200, Loss: 2233.829102, Accuracy: 0.936700
Step: 300, Loss: 1998.636230, Accuracy: 0.941400
Step: 400, Loss: 1890.096558, Accuracy: 0.945000
Step: 500, Loss: 1749.049316, Accuracy: 0.944700
Step: 600, Loss: 1395.699463, Accuracy: 0.957300
Step: 700, Loss: 1350.074951, Accuracy: 0.959800
Step: 800, Loss: 1298.422119, Accuracy: 0.960200
Step: 900, Loss: 1187.688965, Accuracy: 0.965500
Step: 1000, Loss: 1060.380859, Accuracy: 0.967000
Step: 1100, Loss: 1205.623291, Accuracy: 0.963700
Step: 1200, Loss: 1064.372437, Accuracy: 0.968200
Step: 1300, Loss: 1159.237061, Accuracy: 0.965500
Step: 1400, Loss: 1035.038330, Accuracy: 0.967600
Step: 1500, Loss: 1027.867432, Accuracy: 0.968700
Step: 1600, Loss: 1075.445679, Accuracy: 0.967400
Step: 1700, Loss: 982.560669, Accuracy: 0.968400
Step: 1800, Loss: 950.796997, Accuracy: 0.972500
Step: 1900, Loss: 1018.834534, Accuracy: 0.971500
Step: 2000, Loss: 931.307495, Accuracy: 0.973100


# TensorBoardによる可視化
### terminal上で ```tensorboard --logdir=./mnist_sl_logs```
起動後 http://localhost:6006 にアクセス